In [2]:
import pandas as pd
import datetime 
from slugify import slugify
import numpy as np

data_dir = '../data/'
now = datetime.datetime.now()

def __my_flatten_cols(self, how="_".join, reset_index=True):
    how = (lambda iter: list(iter)[-1]) if how == "last" else how
    self.columns = [how(filter(None, map(str, levels))) for levels in self.columns.values] \
                    if isinstance(self.columns, pd.MultiIndex) else self.columns
    return self.reset_index() if reset_index else self
pd.DataFrame.my_flatten_cols = __my_flatten_cols

In [3]:
#Mine-specific data
mine_data = pd.read_csv(data_dir + 'msha_mine_20190209-0.csv', escapechar='\\')
       
mine_rename = {'controller_id':'curr_ctrlr_id', 'controller_nm':'curr_ctrlr_nm', 
                'oper_id':'curr_oper_id', 'oper_nm':'curr_oper_nm'}
                
mine_data = mine_data.rename(index=str, columns=mine_rename)

In [16]:
#Delinquency data
delinquency_data = pd.read_csv(data_dir + 'debtbyage_20181205_REFINED.csv')
delinquency_data['Delinquent Date'] =  pd.to_datetime(delinquency_data['Delinquent Date'], 
                                                      format='%m/%d/%Y')
delinquency_data['year'] = pd.DatetimeIndex(delinquency_data['Delinquent Date']).year
delinquency_data['month'] = pd.DatetimeIndex(delinquency_data['Delinquent Date']).month

#because this delinquency data is fucked, we're going to remove everything that
#isn't related to delinquencies and then do our catod and current joins
delinquency_data = delinquency_data[['Assess Case Nbr','Mine ID','Violator ID','Violator Name', 
                                     'Violator Type','Controller Id','Controller Name','Age Category',
                                     'Delinquent Date','Delinquent Type','Ending Balance','year','month']]

delinquency_data = delinquency_data.rename(index=str, columns={'Assess Case Nbr':'ass_case_nbr',
                                                              'Mine ID':'mine_id',
                                                              'Violator ID':'violator_id',
                                                              'Violator Name':'violator_nm',
                                                              'Controller Id':'dba_ctrlr_id',
                                                              'Controller Name':'dba_ctrlr_nm',
                                                              'Violator Type':'violator_type',
                                                              'Age Category':'age_cateogry',
                                                              'Delinquent Date':'delinquent_date',
                                                              'Delinquent Type':'delinquent_type',
                                                              'Ending Balance':'ending_balance'})

dba_ctrlr_list = delinquency_data[['ass_case_nbr','dba_ctrlr_id','dba_ctrlr_nm']].drop_duplicates()

earliest_dict = delinquency_data.groupby('mine_id')['delinquent_date'].agg('min').to_dict()
earliest_series = delinquency_data.groupby('mine_id')['delinquent_date'].agg('min')
earliest_df = pd.DataFrame({'mine_id':earliest_series.index, 'earliest_date':earliest_series.values})  
#Need to round earliest_date up to the next year so we can factor only years with full delinquency
earliest_df['earliest_year'] = earliest_df['earliest_date'].map(lambda x: x.year+1)

mine_del_summary = delinquency_data.groupby('mine_id').agg({'delinquent_date':['max','min'],
                                                            'ending_balance':['sum','count']}).my_flatten_cols()
mine_del_summary = mine_del_summary.rename(index=str, columns={'delinquent_date_max':'latest_del_date',
                                                               'delinquent_date_min':'earliest_del_date',
                                                               'ending_balance_sum':'total_due',
                                                               'ending_balance_count':'num_del_records'})

del_data_curr_join = curr_mine_data_merge(delinquency_data)
del_data_curr_catod_reduced = catod_mine_data_merge(del_data_curr_join)
del_data_curr_catod = curr_mine_data_merge(del_data_curr_catod_reduced)
del_data_joined = del_data_curr_catod.merge(dba_ctrlr_list, 
                                            on='ass_case_nbr', how='left', suffixes=('_d','c'))
del_data_joined = del_data_joined[['catod_oper_id', 'catod_ctrlr_id', 'catod_ctrlr_start_dt',
       'catod_ctrlr_end_dt', 'catod_oper_nm', 'ass_case_nbr',
       'mine_id', 'delinquent_date', 'ending_balance',
       'curr_mine_nm', 'c_m_ind', 'mine_type_cd', 'curr_stat_cd',
       'curr_stat_dt', 'curr_ctrlr_id_m', 'curr_ctrlr_nm', 'curr_oper_id',
       'curr_oper_nm', 'state_abbr', 'fips_cnty_nm', 'curr_ownr_beg_dt',
       'dba_ctrlr_id', 'dba_ctrlr_nm']]
del_data_joined = del_data_joined.rename(index=str, columns={'curr_ctrlr_id_m':'curr_ctrlr_id'})




In [5]:
#Controller-Operation relationship data
cntrlr_hist_data = pd.read_csv(data_dir + 'msha_controller_history_20190105-0.csv')
cntrlr_hist_data['ctrlr_end_dt'] = cntrlr_hist_data['ctrlr_end_dt'].fillna('2019-01-01')
cntrlr_hist_data['ctrlr_start_dt'] =  pd.to_datetime(cntrlr_hist_data['ctrlr_start_dt'], 
                                                      format='%Y-%m-%d')
cntrlr_hist_data['ctrlr_end_dt'] =  pd.to_datetime(cntrlr_hist_data['ctrlr_end_dt'], 
                                                      format='%Y-%m-%d')

In [6]:
ass_violations = pd.read_csv(data_dir + 'from_msha/AssessedViolations.txt', 
                         sep='|', 
                         encoding = 'ISO-8859-1',
                         dtype={'ASSESS_CASE_NO': np.object_})

ass_violations['DELINQUENT_DT'] = pd.to_datetime(ass_violations['DELINQUENT_DT'], 
                                                      format='%m/%d/%Y')

/Users/akanik/.virtualenvs/code-FVmSw4CU/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
violations = pd.read_csv(data_dir + 'from_msha/Violations.txt', 
                         sep='|', 
                         encoding = 'ISO-8859-1')
violations['VIOLATION_OCCUR_DT'] = pd.to_datetime(violations['VIOLATION_OCCUR_DT'], 
                                                      format='%m/%d/%Y')
violations['VIOLATION_ISSUE_DT'] = pd.to_datetime(violations['VIOLATION_ISSUE_DT'], 
                                                      format='%m/%d/%Y')
violations['delinquent_date'] = violations['MINE_ID'].map(earliest_series)
violations['delinquent'] = np.where(violations['VIOLATION_ISSUE_DT'] > violations['delinquent_date'],1,0)

/Users/akanik/.virtualenvs/code-FVmSw4CU/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,13,25,27,34,36,37,46,55,56,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
accidents = pd.read_csv(data_dir + 'from_msha/Accidents.txt', 
                         sep='|', 
                         encoding = 'ISO-8859-1')
accidents['ACCIDENT_DT'] = pd.to_datetime(accidents['ACCIDENT_DT'], 
                                                      format='%m/%d/%Y')

not_contractor = pd.isna(accidents['CONTRACTOR_ID'])
not_natural = accidents['DEGREE_INJURY_CD'].isin(['01','02','03','04','05','06','07','10'])
injuries = accidents.loc[(not_contractor) & (not_natural)]

injuries['delinquent_date'] = injuries['MINE_ID'].map(earliest_series)
injuries['delinquent'] = np.where(injuries['ACCIDENT_DT'] > injuries['delinquent_date'],1,0)

/Users/akanik/.virtualenvs/code-FVmSw4CU/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/akanik/.virtualenvs/code-FVmSw4CU/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [9]:
hrs_data = pd.read_csv(data_dir + 'from_msha/MinesProdYearly.txt', 
                         sep='|', 
                         encoding = 'ISO-8859-1')

In [10]:
#Create a unique list of currently delinquent mines
def get_delinquent_mines():
    return delinquency_data['mine_id'].unique()

#Create a unique list of mines that have been delinquent since the year passed to this function
def get_delinquent_since(year): 
    delinquent_since = earliest_df[earliest_df['earliest_year'] <= year]
    return delinquent_since['mine_id'].unique()

In [11]:
def curr_mine_data_merge(mine_level_agg):
    
    keep_cols = ['mine_id','curr_mine_nm', 'c_m_ind', 'mine_type_cd', 'curr_stat_cd',
           'curr_stat_dt', 'curr_ctrlr_id', 'curr_ctrlr_nm', 'curr_oper_id',
           'curr_oper_nm', 'state_abbr', 'fips_cnty_nm', 'curr_ownr_beg_dt']
    mine_data_refined = mine_data[keep_cols]
    
    agg_mine = pd.merge(mine_level_agg, mine_data_refined, 
                        how='left', on='mine_id', suffixes=('_a','_m'))
    #agg_mine_del = pd.merge(agg_mine, mine_del_summary, 
    #                        how='left', on='mine_id', suffixes=('_am','_d'))

    return agg_mine

In [12]:
def catod_mine_data_merge(delinquency_data):
    catod_del_merge = pd.merge(cntrlr_hist_data,delinquency_data,how='left',left_on='oper_id',right_on='violator_id')

    #Filters for the filtering we'll be doing on catod_del_merge
    gt_dates = catod_del_merge['delinquent_date'] >= catod_del_merge['ctrlr_start_dt']
    lt_dates = catod_del_merge['delinquent_date'] <= catod_del_merge['ctrlr_end_dt']

    #So here's where we drop the records where the dates don't match up
    catod_filtered = catod_del_merge[(gt_dates) & (lt_dates)]

    #We're getting rid of columns so that we can drop duplicates and get back to our original
    #delinquency_data count
    catod_reduced = catod_filtered[['oper_id','ctrlr_id','ctrlr_start_dt','ctrlr_end_dt','oper_nm',
                                    'ass_case_nbr','curr_ctrlr_id','mine_id','delinquent_date',
                                    'ending_balance']].drop_duplicates()

    catod_reduced = catod_reduced.rename(index=str, columns={'oper_id':'catod_oper_id',
                                                             'ctrlr_id':'catod_ctrlr_id',
                                                             'ctrlr_start_dt':'catod_ctrlr_start_dt',
                                                             'ctrlr_end_dt':'catod_ctrlr_end_dt',
                                                             'oper_nm':'catod_oper_nm'})
    return catod_reduced

In [13]:
def calc_injury_rate(row):    
    if row['ANNUAL_HOURS'] < 2000:
        return 0
    else:
        return (row['inj_cnt'] / (row['ANNUAL_HOURS'] / 2000)) * 100

def get_range_injury_rate(inj_data, hrs_data, rate_type, year_list):

    inj_year = inj_data.groupby('CAL_YR')['DOCUMENT_NO'].count().reset_index()
    inj_year = inj_year.rename(index=str, columns={'DOCUMENT_NO': 'inj_cnt'})

    hrs_year = hrs_data.groupby('CAL_YR', as_index=False)['ANNUAL_HOURS'].sum()

    hrs_inj_year = pd.merge(hrs_year, inj_year, 
                                      how='left', on=['CAL_YR'], suffixes=('_m','_inj'))

    hrs_inj_year['inj_rate'] = hrs_inj_year.apply(calc_injury_rate, axis=1)

    avg_inj_rate = hrs_inj_year[hrs_inj_year['CAL_YR'].isin(year_list)]['inj_rate'].mean()
    
    return rate_type + ': ' + str(round(avg_inj_rate, 2))

In [14]:
###### DEBT #########
#Debt based on mines they currently own
def get_curr_debt(ctrlr_id):
    return del_data_joined[del_data_joined['curr_ctrlr_id'].isin(ctrlr_id)]['ending_balance'].sum()

#Debt based on controller history controller at time of delinquency
def get_orig_debt(ctrlr_id):
    return del_data_joined[del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)]['ending_balance'].sum()

#Debt based on debt by age dataset controller column
def get_dba_debt(ctrlr_id):
    return del_data_joined[del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)]['ending_balance'].sum()

#Debt for which they are listed as controller in controller history and in DBA
def get_orig_dba_debt(ctrlr_id):
    orig_debt = del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)
    dba_debt = del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)
    return del_data_joined[(orig_debt) & (dba_debt)]['ending_balance'].sum()

#Debt for which they are listed as controller in controller history but 
#NOT as controller in DBA
def get_orig_no_dba_debt(ctrlr_id):
    orig_debt = del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)
    dba_debt = del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)
    return del_data_joined[(orig_debt) & (~dba_debt)]['ending_balance'].sum()

#Debt for which they are NOT listed as controller in controller history but 
#ARE listed as controller in DBA
def get_no_orig_dba_debt(ctrlr_id):
    orig_debt = del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)
    dba_debt = del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)
    return del_data_joined[(~orig_debt) & (dba_debt)]['ending_balance'].sum()



###### MINES #########
#Mine count based on mines they currently own
def get_curr_mine_count(ctrlr_id):
    return del_data_joined[del_data_joined['curr_ctrlr_id'].isin(ctrlr_id)]['mine_id'].nunique()

#Mine count based on controller history controller at time of delinquency
def get_orig_mine_count(ctrlr_id):
    return del_data_joined[del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)]['mine_id'].nunique()

#Mine count based on debt by age dataset controller column
def get_dba_mine_count(ctrlr_id):
    return del_data_joined[del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)]['mine_id'].nunique()

#Mine count for which they are listed as controller in controller history but 
#NOT as controller in DBA
def get_orig_no_dba_mine_count(ctrlr_id):
    orig_mines = del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)
    dba_mines = del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)
    return del_data_joined[(orig_mines) & (~dba_mines)]['mine_id'].nunique()



###### RECORDS #########
#Delinquency record count based on mines they currently own
def get_curr_record_count(ctrlr_id):
    return del_data_joined[del_data_joined['curr_ctrlr_id'].isin(ctrlr_id)]['ass_case_nbr'].nunique()

#Delinquency record count based on controller history controller at time of delinquency
def get_orig_record_count(ctrlr_id):
    return del_data_joined[del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)]['ass_case_nbr'].nunique()

#Delinquency record count based on debt by age dataset controller column
def get_dba_record_count(ctrlr_id):
    return del_data_joined[del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)]['ass_case_nbr'].nunique()

#Delinquency records list based on debt by age dataset controller column
def get_dba_records(ctrlr_id):
    return del_data_joined[del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)]['ass_case_nbr'].unique()


###### DATA #########
def get_curr_data(ctrlr_id):
    return del_data_joined[del_data_joined['curr_ctrlr_id'].isin(ctrlr_id)]

#Delinquency data based for which they are listed as controller in DBA
def get_dba_data(ctrlr_id):
    return del_data_joined[del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)]

#Delinquency data for which they are listed as controller in controller history and in DBA
def get_orig_dba_data(ctrlr_id):
    orig_debt = del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)
    dba_debt = del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)
    return del_data_joined[(orig_debt) & (dba_debt)]

#Delinquency data for which they are listed as controller in controller history but 
#NOT as controller in DBA
def get_orig_no_dba_data(ctrlr_id):
    orig_mines = del_data_joined['catod_ctrlr_id'].isin(ctrlr_id)
    dba_mines = del_data_joined['dba_ctrlr_id'].isin(ctrlr_id)
    return del_data_joined[(orig_mines) & (~dba_mines)]

## Mines Justice Group are responsible for according to DBA

In [44]:
justice_dba_mine_count = get_dba_mine_count(['0091855','C04355'])
print(justice_dba_mine_count)

71


In [518]:
justice_dba_data = get_dba_data(['0091855','C04355'])
justice_dba_by_mine = justice_dba_data.groupby('mine_id', as_index=False)['ending_balance'].sum()
justice_dba_by_mine_combo = curr_mine_data_merge(justice_dba_by_mine)
justice_dba_by_mine_combo.to_csv(data_dir + 'analysis/justice/justice-dba-by-mine.csv')

## Amount Justice Group are responsible for according to DBA

In [203]:
total_dba_debt = delinquency_data['ending_balance'].sum()
justice_dba_debt = get_dba_debt(['0091855','C04355'])
justice_jrs_dba_debt = get_dba_debt(['0091855'])
justice_sr_dba_debt = get_dba_debt(['C04355'])

print('Total DBA debt: ${:,.2f}'.format(total_dba_debt))
print('Justice Group: ${:,.2f}'.format(justice_dba_debt))
print('Justice Jrs: ${:,.2f}'.format(justice_jrs_dba_debt))
print('Justice Sr: ${:,.2f}'.format(justice_sr_dba_debt))

Total DBA debt: $45,098,251.75
Justice Group: $4,303,377.27
Justice Jrs: $3,785,194.85
Justice Sr: $518,182.42


## Mines Justice Group was controller at when they received delinquencies

In [46]:
justice_orig_mine_count = get_orig_mine_count(['0091855','C04355'])
print(justice_orig_mine_count)

83


## Amount of current delinquencies from mines where Justice Group was controller at time of delinquency

In [38]:
justice_orig_debt = get_orig_debt(['0091855','C04355'])
print('${:,.2f}'.format(justice_orig_debt))

$5,647,431.86


## Which specific controllers did the Justice Group Originated/DBA debt come from?

In [184]:
justice_orig_dba_data = get_orig_dba_data(['0091855','C04355'])
print(justice_orig_dba_data.columns)
print(justice_orig_dba_data.groupby('catod_ctrlr_id')['ending_balance'].sum())

Index(['catod_oper_id', 'catod_ctrlr_id', 'catod_ctrlr_start_dt',
       'catod_ctrlr_end_dt', 'catod_oper_nm', 'ass_case_nbr', 'mine_id',
       'delinquent_date', 'ending_balance', 'curr_mine_nm', 'c_m_ind',
       'mine_type_cd', 'curr_stat_cd', 'curr_stat_dt', 'curr_ctrlr_id',
       'curr_ctrlr_nm', 'curr_oper_id', 'curr_oper_nm', 'state_abbr',
       'fips_cnty_nm', 'curr_ownr_beg_dt', 'dba_ctrlr_id', 'dba_ctrlr_nm'],
      dtype='object')
catod_ctrlr_id
0091855     761183.07
C04355     3541852.19
Name: ending_balance, dtype: float64


## Amount of current delinquencies that both originated with and is still assigned to the Justice Group

In [180]:
justice_orig_dba_debt = get_orig_dba_debt(['0091855','C04355'])
print('${:,.2f}'.format(justice_orig_dba_debt))

$4,303,035.26


## Mines that Justice Group was responsible for at time of delinquency but is NOT responsible for in the DBA dataset

In [40]:
justice_orig_no_dba_mine_count = get_orig_no_dba_mine_count(['0091855','C04355'])
print(justice_orig_no_dba_mine_count)

12


## Amount that Justice Group was responsible for originating but for which they are NOT being held responsible in DBA

In [41]:
justice_orig_no_dba_debt = get_orig_no_dba_debt(['0091855','C04355'])
print('${:,.2f}'.format(justice_orig_no_dba_debt))

$1,344,396.60


## Which mines are they no longer being held responsible for? Do they still own them now?

In [189]:
justice_orig_no_dba_data = get_orig_no_dba_data(['C04355'])

justice_orig_no_dba_mines_by_ctrlr = justice_orig_no_dba_data.groupby('dba_ctrlr_id').agg({'ending_balance':'sum',
                                                                                          'mine_id':'nunique'})

justice_orig_no_dba_mines_curr_join = curr_mine_data_merge(justice_orig_no_dba_data)

print(justice_orig_no_dba_mines_by_ctrlr)

              ending_balance  mine_id
dba_ctrlr_id                         
0085759           1248521.07        7
0091855           3023669.77       56


## Which mines do the Justices owe the most at?

In [501]:
print(get_dba_data(['0091855','C04355']).columns)

Index(['catod_oper_id', 'catod_ctrlr_id', 'catod_ctrlr_start_dt',
       'catod_ctrlr_end_dt', 'catod_oper_nm', 'ass_case_nbr', 'mine_id',
       'delinquent_date', 'ending_balance', 'curr_mine_nm', 'c_m_ind',
       'mine_type_cd', 'curr_stat_cd', 'curr_stat_dt', 'curr_ctrlr_id',
       'curr_ctrlr_nm', 'curr_oper_id', 'curr_oper_nm', 'state_abbr',
       'fips_cnty_nm', 'curr_ownr_beg_dt', 'dba_ctrlr_id', 'dba_ctrlr_nm'],
      dtype='object')


In [506]:
justice_dba_data = get_dba_data(['0091855','C04355'])
justice_dba_data = justice_dba_data[justice_dba_data['curr_stat_cd'] == 'Active']
justice_dba_by_mine = justice_dba_data.groupby(['mine_id','curr_mine_nm','state_abbr','curr_ctrlr_id'],as_index=False)['ending_balance'].sum().sort_values('ending_balance',ascending=False)
print(justice_dba_by_mine)

     mine_id                   curr_mine_nm state_abbr curr_ctrlr_id  \
0  1518015.0  Bent Mountain Surface Mine #2         KY       0142161   
5  4407160.0           Preacher Creek Strip         VA       0091855   
1  1518947.0     Bull Creek Surface Mine #1         KY       0142161   
7  4609135.0          Coal Mountain Loadout         WV        C15876   
3  4406746.0                  Prep Plant #1         VA        C13088   
2  4406544.0          AG4 Sawmill Hollow #2         VA       0142161   
6  4604734.0                Orchard Loadout         WV       0091855   
4  4407115.0                       Baden #1         VA       0091855   

   ending_balance  
0        21441.54  
5        13953.90  
1         3970.91  
7         3370.97  
3         2751.90  
2         2434.51  
6         2281.71  
4          716.54  


## Scripting violation database

In [365]:
def get_dba_ctrlr_violations(ctrlr_id):
    dba_record_list = get_dba_records(ctrlr_id)
    
    dba_records = del_data_joined[del_data_joined['ass_case_nbr'].isin(dba_record_list)]
    dba_ass_viol = ass_violations[ass_violations['ASSESS_CASE_NO'].isin(dba_record_list)]
    
    ass_viol_grouped = dba_ass_viol.groupby(['ASSESS_CASE_NO','DELINQUENT_DT']).agg({'PROPOSED_PENALTY_AMT':'sum',
                                                                                     'CURRENT_ASSESSMENT_AMT':'sum',
                                                                                     'MINEACT_INTEREST_AMT':'sum',
                                                                                     'EXLATE_INTEREST_AMT':'sum',
                                                                                     'PAID_PROPOSED_PENALTY_AMT':'sum',
                                                                                     'PAID_MINEACT_INTEREST_AMT':'sum',
                                                                                     'PAID_EXLATE_INTEREST_AMT':'sum',}).reset_index()
    
    dba_viol_join = dba_records.merge(ass_viol_grouped,
                                      how='left',
                                      left_on=['ass_case_nbr','delinquent_date'],
                                      right_on=['ASSESS_CASE_NO','DELINQUENT_DT'])
    
    return dba_viol_join
    
def print_dba_ctrlr_violations_summary(dba_viol_join, ctrlr_name):
    
    total_proposed = dba_viol_join.PROPOSED_PENALTY_AMT.sum()
    total_ass_amt = dba_viol_join.CURRENT_ASSESSMENT_AMT.sum()
    total_mineact_amt = dba_viol_join.MINEACT_INTEREST_AMT.sum()
    total_exlate_amt = dba_viol_join.EXLATE_INTEREST_AMT.sum()
    total_penalty_paid = dba_viol_join.PAID_PROPOSED_PENALTY_AMT.sum()
    total_mineact_paid = dba_viol_join.PAID_MINEACT_INTEREST_AMT.sum()
    total_exlate_paid = dba_viol_join.PAID_EXLATE_INTEREST_AMT.sum()
    total_ending_balance = dba_viol_join.ending_balance.sum()

    total_charges = total_ass_amt + total_mineact_amt + total_exlate_amt
    total_paid = total_penalty_paid + total_mineact_paid + total_exlate_paid
    total_owed = total_charges - total_paid
    dba_viol_diff = total_ending_balance - total_owed
    
    print('***According to MSHA Violations records***')
    print(ctrlr_name + ' has been charged: ' + '${:,.2f}'.format(total_charges))
    print(ctrlr_name + ' has paid on those charges: ' + '${:,.2f}'.format(total_paid))
    print(ctrlr_name + ' still owes: ' + '${:,.2f}'.format(total_owed))
    print('')
    print('The Debt by Age data say ' + ctrlr_name + ' owes: ' + '${:,.2f}'.format(total_ending_balance))
    print('The violations data say ' + ctrlr_name + ' owes: ' + '${:,.2f}'.format(total_owed))
    if dba_viol_diff > 0:
        print('The difference (' + '${:,.2f}'.format(dba_viol_diff) + 
              ') could be the result of them paying off some of their debts between Dec. 2018 and now.')
    elif dba_viol_diff < 0:
        print('The difference (' + '${:,.2f}'.format(dba_viol_diff) + 
              ') could be the result of them not paying any of their debts between Dec. 2018 and now.')
    

## Amount of debt paid down by Justice Group according to MSHA violations database and the difference between violations database totals and BDA totals


In [366]:
justice_violations = get_dba_ctrlr_violations(['0091855','C04355'])
print_dba_ctrlr_violations_summary(justice_violations, 'Justice Group')

***According to MSHA Violations records***
Justice Group has been charged: $4,738,454.02
Justice Group has paid on those charges: $475,320.23
Justice Group still owes: $4,263,133.79

The Debt by Age data say Justice Group owes: $4,303,377.27
The violations data say Justice Group owes: $4,263,133.79
The difference ($40,243.48) could be the result of them paying off some of their debts between Dec. 2018 and now.


The above "still owed" amount is different than the ending balance we find in our DBA. Upon doing a manual comparison of violation data and DBA data, it appears that there are some assessment case number / delinquent date combos that appear in the DBA data that do not appear in the violations data. 

I can think of a few reasons for this:
- the dates were slightly off and thus they did not merge properly
- the debts were paid off since we received our December 2018 DBA data

I hand checked a couple of the assessment case numbers that didn't match and they don't appear to be in the violations dataset. So I'm leaning toward my second theory on this one.

So I think we should determine the difference between the DBA ending_balance and the violations "still owes" and mention that the records indicate that they might have paid these violations off completely.

## Jeffrey Hoops, Revelation Energy Violations DBA analysis

In [517]:

hoops_data = get_dba_data(['C15863'])
print(hoops_data.mine_id.nunique())
#hoops_data['del_year'] = pd.DatetimeIndex(hoops_data['delinquent_date']).year
#print(hoops_data.groupby('del_year')['ass_case_nbr'].nunique())
hoops_violations = get_dba_ctrlr_violations(['C15863'])
print_dba_ctrlr_violations_summary(hoops_violations, 'Jeffrey Hoops, Revelation Energy')

50
***According to MSHA Violations records***
Jeffrey Hoops, Revelation Energy has been charged: $1,047,150.32
Jeffrey Hoops, Revelation Energy has paid on those charges: $93,202.65
Jeffrey Hoops, Revelation Energy still owes: $953,947.67

The Debt by Age data say Jeffrey Hoops, Revelation Energy owes: $926,471.69
The violations data say Jeffrey Hoops, Revelation Energy owes: $953,947.67
The difference ($-27,475.98) could be the result of them not paying any of their debts between Dec. 2018 and now.


## Virginia Conservation Legacy Fund Violations DBA analysis

In [368]:
vclf_violations = get_dba_ctrlr_violations(['0121101'])
print_dba_ctrlr_violations_summary(vclf_violations, 'Virginia Conservation Legacy Fund')

***According to MSHA Violations records***
Virginia Conservation Legacy Fund has been charged: $735,316.32
Virginia Conservation Legacy Fund has paid on those charges: $0.00
Virginia Conservation Legacy Fund still owes: $735,316.32

The Debt by Age data say Virginia Conservation Legacy Fund owes: $736,348.94
The violations data say Virginia Conservation Legacy Fund owes: $735,316.32
The difference ($1,032.62) could be the result of them paying off some of their debts between Dec. 2018 and now.


## James H Booth Violations DBA analysis

In [369]:
booth_violations = get_dba_ctrlr_violations(['C11194'])
print_dba_ctrlr_violations_summary(booth_violations, 'James H Booth')

***According to MSHA Violations records***
James H Booth has been charged: $1,101,490.28
James H Booth has paid on those charges: $588,075.21
James H Booth still owes: $513,415.07

The Debt by Age data say James H Booth owes: $582,219.63
The violations data say James H Booth owes: $513,415.07
The difference ($68,804.56) could be the result of them paying off some of their debts between Dec. 2018 and now.


## Top debtors
- We'll want money owed according to DBA
- Mine count according to DBA
- Amount paid down according to MSHA violations db

In [563]:
top_debtors = del_data_joined.groupby(['dba_ctrlr_id','dba_ctrlr_nm']).agg({'ending_balance':'sum',
                                                        'mine_id':'nunique'}).sort_values('ending_balance', ascending=False).reset_index()

#print(delinquency_data.dba_ctrlr_nm.nunique())
print(top_debtors)

2436
     dba_ctrlr_id                                  dba_ctrlr_nm  \
0         0091855      James C  Justice lll; Jillian L  Justice   
1         0085759                                    Mechel Oao   
2         0086815           Gary J  Ronald; Archibald C  Parker   
3          C15863                              Jeffery A  Hoops   
4          C05558                                    Carl  Kirk   
5         0121101      Virginia Conservation Legacy Fund (VCLF)   
6         0046489                                Timothy R  Dye   
7          C11194                                James H  Booth   
8          C00085                            Richard H  Abraham   
9          C04355                           James C  Justice II   
10         C15593                             CraKol Energy LLC   
11        0069491                               Jody D  Puckett   
12        0104813                     Estate of Ernest E Varney   
13        0130321                      ERP Compliant Fuel

In [17]:
abrahams_debt = del_data_joined[del_data_joined['dba_ctrlr_id'] == 'C00085']
print(abrahams_debt)

      catod_oper_id catod_ctrlr_id catod_ctrlr_start_dt catod_ctrlr_end_dt  \
12815        P24562         C00085           2006-10-02         2019-01-01   
12816        P24562         C00085           2006-10-02         2019-01-01   
12817        P24562         C00085           2006-10-02         2019-01-01   
12818        P24562         C00085           2006-10-02         2019-01-01   
12819        P24562         C00085           2006-10-02         2019-01-01   
12820        P24562         C00085           2006-10-02         2019-01-01   
12821        P24562         C00085           2006-10-02         2019-01-01   
12822        P24562         C00085           2006-10-02         2019-01-01   
12823        P24562         C00085           2006-10-02         2019-01-01   
12824        P24562         C00085           2006-10-02         2019-01-01   
12825        P24562         C00085           2006-10-02         2019-01-01   
12826        P24562         C00085           2006-10-02         

## Top regional debtors
- We'll want money owed according to DBA
- Mine count according to DBA
- Amount paid down according to MSHA violations db

In [338]:
#Virginia Conservation Legacy Fund (0121101) wants to see what they owe
#Let's export their slice of the data for them.

vclf_dba_data = get_dba_data(['0121101'])
vclf_dba_debt = get_dba_debt(['0121101'])
vclf_dba_mines = get_dba_mine_count(['0121101'])

print(vclf_dba_debt)
print(vclf_dba_mines)
#vclf_dba_data.to_csv(data_dir + 'analysis/by_controller/20181205_vclf_debtbyage_data.csv')

736348.9400000001
2


## General accident rates for delinquent and nondelinquent mines

In [287]:
def filter_mines(mine_data, c_m, mine_type, mine_status, del_status, hr_text):
    if c_m:
        filter_cm = mine_data[mine_data['c_m_ind'] == c_m]['mine_id'].unique()
    else:
        filter_cm = mine_data['mine_id'].unique()
    if mine_type:
        filter_mine_type = mine_data[mine_data['mine_type_cd'] == mine_type]['mine_id'].unique()
    else:
        filter_mine_type = mine_data['mine_id'].unique()
    if mine_status:
        filter_mine_status = mine_data[mine_data['curr_stat_cd'] == mine_status]['mine_id'].unique()
    else:
        filter_mine_status = mine_data['mine_id'].unique()
    
    filter_injuries = injuries[(injuries['MINE_ID'].isin(filter_cm)) & (injuries['MINE_ID'].isin(filter_mine_type)) & (injuries['MINE_ID'].isin(filter_mine_status))]
    filter_hours = hrs_data[(hrs_data['MINE_ID'].isin(filter_cm)) & (hrs_data['MINE_ID'].isin(filter_mine_type)) & (hrs_data['MINE_ID'].isin(filter_mine_status))]
    
    if del_status == 'Delinquent':
        filter_injuries = filter_injuries[filter_injuries['MINE_ID'].isin(get_delinquent_since(2014))]
        filter_hours = filter_hours[filter_hours['MINE_ID'].isin(get_delinquent_since(2014))]
    elif del_status == 'Nondelinquent':
        filter_injuries = filter_injuries[~filter_injuries['MINE_ID'].isin(get_delinquent_mines())]
        filter_hours = filter_hours[~filter_hours['MINE_ID'].isin(get_delinquent_mines())]
    elif del_status == 'All':
        filter_injuries = filter_injuries
        filter_hours = filter_hours
    
    #10-year average injury rate
    print(get_range_injury_rate(filter_injuries, filter_hours, hr_text, list(range(2009,2019,1))))
    #5-year average injury rate
    #print(get_range_injury_rate(injuries, hours, hr_text, list(range(2014,2019,1))))
    #ALL-year average injury rate
    #print(get_range_injury_rate(injuries, hours, hr_text, list(range(1994,2019,1))))
    

In [309]:
print('*****Avg Injury Rates For All Mines*****')
filter_mines(mine_data,'', '', '', 'All', 'All mines')
filter_mines(mine_data,'', '', '', 'Delinquent', 'All delinquent mines')
filter_mines(mine_data,'', '', '', 'Nondelinquent', 'All nondelinquent mines')
filter_mines(mine_data,'', '', 'Active', 'All', 'Active mines')
filter_mines(mine_data,'', '', 'Active', 'Delinquent', 'Active, delinquent mines')
filter_mines(mine_data,'', '', 'Active', 'Nondelinquent', 'Active, nondelinquent mines')
print('')
print('*****Avg Injury Rates For Coal Mines*****')
filter_mines(mine_data,'C', '', '', 'All', 'All coal mines')
filter_mines(mine_data,'C', '', '', 'Delinquent', 'All delinquent coal mines')
filter_mines(mine_data,'C', '', '', 'Nondelinquent', 'All nondelinquent coal mines')
filter_mines(mine_data,'C', '', 'Active', 'All', 'Active coal mines')
filter_mines(mine_data,'C', '', 'Active', 'Delinquent', 'Active, delinquent coal mines')
filter_mines(mine_data,'C', '', 'Active', 'Nondelinquent', 'Active, nondelinquent coal mines')
print('')
filter_mines(mine_data,'C', 'Underground', '', 'All', 'All underground coal mines')
filter_mines(mine_data,'C', 'Underground', '', 'Delinquent', 'All delinquent, underground coal mines')
filter_mines(mine_data,'C', 'Underground', '', 'Nondelinquent', 'All nondelinquent, underground coal mines')
filter_mines(mine_data,'C', 'Underground', 'Active', 'All', 'Active underground coal mines')
filter_mines(mine_data,'C', 'Underground', 'Active', 'Delinquent', 'Active, delinquent, underground coal mines')
filter_mines(mine_data,'C', 'Underground', 'Active', 'Nondelinquent', 'Active, nondelinquent, underground coal mines')
print('')
filter_mines(mine_data,'C', 'Surface', '', 'All', 'All surface coal mines')
filter_mines(mine_data,'C', 'Surface', '', 'Delinquent', 'All delinquent, surface coal mines')
filter_mines(mine_data,'C', 'Surface', '', 'Nondelinquent', 'All nondelinquent, surface coal mines')
filter_mines(mine_data,'C', 'Surface', 'Active', 'All', 'Active surface coal mines')
filter_mines(mine_data,'C', 'Surface', 'Active', 'Delinquent', 'Active, delinquent, surface coal mines')
filter_mines(mine_data,'C', 'Surface', 'Active', 'Nondelinquent', 'Active, nondelinquent, surface coal mines')
print('')
filter_mines(mine_data,'C', 'Facility', '', 'All', 'All facility coal mines')
filter_mines(mine_data,'C', 'Facility', '', 'Delinquent', 'All delinquent, facility coal mines')
filter_mines(mine_data,'C', 'Facility', '', 'Nondelinquent', 'All nondelinquent, facility coal mines')
filter_mines(mine_data,'C', 'Facility', 'Active', 'All', 'Active facility coal mines')
filter_mines(mine_data,'C', 'Facility', 'Active', 'Delinquent', 'Active, delinquent, facility coal mines')
filter_mines(mine_data,'C', 'Facility', 'Active', 'Nondelinquent', 'Active, nondelinquent, facility coal mines')

*****Avg Injury Rates For All Mines*****
All mines: 2.88
All delinquent mines: 3.65
All nondelinquent mines: 2.66
Active mines: 2.68
Active, delinquent mines: 3.12
Active, nondelinquent mines: 2.51

*****Avg Injury Rates For Coal Mines*****
All coal mines: 3.83
All delinquent coal mines: 4.45
All nondelinquent coal mines: 3.38
Active coal mines: 3.38
Active, delinquent coal mines: 3.32
Active, nondelinquent coal mines: 2.98

All underground coal mines: 5.41
All delinquent, underground coal mines: 6.5
All nondelinquent, underground coal mines: 4.97
Active underground coal mines: 4.92
Active, delinquent, underground coal mines: 4.83
Active, nondelinquent, underground coal mines: 4.57

All surface coal mines: 1.77
All delinquent, surface coal mines: 2.94
All nondelinquent, surface coal mines: 1.67
Active surface coal mines: 1.4
Active, delinquent, surface coal mines: 2.68
Active, nondelinquent, surface coal mines: 1.31

All facility coal mines: 3.77
All delinquent, facility coal mines: 3.

## Fact check Justice Group statement
"The Justice Group is also very proud to report that its accident and incident history of currently operated mines is very favorable."

In [319]:
justice_curr_del_mines = get_curr_data(['0091855','C04355'])
justice_active_curr_del_mines = justice_curr_del_mines[justice_curr_del_mines['curr_stat_cd'] == 'Active']
print(justice_active_curr_del_mines.groupby(['mine_id','mine_type_cd'])['mine_id'].nunique())
#print(justice_active_curr_del_mines.columns)
#justice_active_current_mines = justice_current_mines[justice_current_mines['curr_stat_cd'] == 'Active']
#print(justice_active_current_mines.mine_id.unique())

mine_id    mine_type_cd
4407115.0  Surface         1
4407160.0  Surface         1
4604734.0  Facility        1
4606578.0  Surface         1
4608786.0  Underground     1
4608884.0  Underground     1
4609239.0  Surface         1
4609316.0  Facility        1
Name: mine_id, dtype: int64


In [324]:
#print(hrs_data[hrs_data['MINE_ID'] == 4604734])
print(injuries[injuries['MINE_ID'] == 4608786])
#print(injuries.columns)

        MINE_ID CONTROLLER_ID                           CONTROLLER_NAME  \
10394   4608786       0091855  Jillean L  Justice; James C  Justice lll   
39088   4608786       0091855  Jillean L  Justice; James C  Justice lll   
140030  4608786       0091855  Jillean L  Justice; James C  Justice lll   

       OPERATOR_ID              OPERATOR_NAME CONTRACTOR_ID   DOCUMENT_NO  \
10394      0095328  Nufac Mining Company, Inc           NaN  220172510016   
39088      0095328  Nufac Mining Company, Inc           NaN  220172370006   
140030     0095328  Nufac Mining Company, Inc           NaN  220181620044   

        SUBUNIT_CD      SUBUNIT ACCIDENT_DT  ...  TRANS_TERM  \
10394            1  UNDERGROUND  2017-08-28  ...           N   
39088            1  UNDERGROUND  2017-08-14  ...           Y   
140030           1  UNDERGROUND  2018-06-01  ...           N   

        RETURN_TO_WORK_DT  IMMED_NOTIFY_CD    IMMED_NOTIFY  INVEST_BEGIN_DT  \
10394          09/05/2017               ?   NO VALUE F

In [316]:
justice_current_mines = mine_data[mine_data['curr_ctrlr_id'].isin(['0091855','C04355'])]
#since all justice mines are coal mines, we'll just be comparing them against national
#coal mine stats

print('*****All Coal Mines*****')
filter_mines(mine_data,'C', '', '', 'All', 'All coal mines')
filter_mines(mine_data,'C', '', '', 'Delinquent', 'All delinquent coal mines')
filter_mines(mine_data,'C', '', '', 'Nondelinquent', 'All nondelinquent coal mines')
print('')
filter_mines(justice_current_mines,'C', '', '', 'All', 'Justice coal mines')
filter_mines(justice_current_mines,'C', '', '', 'Delinquent', 'Justice delinquent coal mines')
filter_mines(justice_current_mines,'C', '', '', 'Nondelinquent', 'Justice nondelinquent coal mines')

print('')

print('*****Underground Coal Mines*****')
filter_mines(mine_data,'C', 'Underground', '', 'All', 'All underground coal mines')
filter_mines(mine_data,'C', 'Underground', '', 'Delinquent', 'All delinquent, underground coal mines')
filter_mines(mine_data,'C', 'Underground', '', 'Nondelinquent', 'All nondelinquent, underground coal mines')
print('')
filter_mines(justice_current_mines,'C', 'Underground', '', 'All', 'Justice underground coal mines')
filter_mines(justice_current_mines,'C', 'Underground', '', 'Delinquent', 'Justice delinquent, underground coal mines')
filter_mines(justice_current_mines,'C', 'Underground', '', 'Nondelinquent', 'Justice nondelinquent, underground coal mines')

print('')

print('*****Surface Coal Mines*****')
filter_mines(mine_data,'C', 'Surface', '', 'All', 'All surface coal mines')
filter_mines(mine_data,'C', 'Surface', '', 'Delinquent', 'All delinquent, surface coal mines')
filter_mines(mine_data,'C', 'Surface', '', 'Nondelinquent', 'All nondelinquent, surface coal mines')
print('')
filter_mines(justice_current_mines,'C', 'Surface', '', 'All', 'Justice surface coal mines')
filter_mines(justice_current_mines,'C', 'Surface', '', 'Delinquent', 'Justice delinquent, surface coal mines')
filter_mines(justice_current_mines,'C', 'Surface', '', 'Nondelinquent', 'Justice nondelinquent, surface coal mines')

print('')

print('*****Facility Coal Mines*****')
filter_mines(mine_data,'C', 'Facility', '', 'All', 'All facility coal mines')
filter_mines(mine_data,'C', 'Facility', '', 'Delinquent', 'All delinquent, facility coal mines')
filter_mines(mine_data,'C', 'Facility', '', 'Nondelinquent', 'All nondelinquent, facility coal mines')
print('')
filter_mines(justice_current_mines,'C', 'Facility', '', 'All', 'Justice facility coal mines')
filter_mines(justice_current_mines,'C', 'Facility', '', 'Delinquent', 'Justice delinquent, facility coal mines')
filter_mines(justice_current_mines,'C', 'Facility', '', 'Nondelinquent', 'Justice nondelinquent, facility coal mines')


*****All Coal Mines*****
All coal mines: 3.83
All delinquent coal mines: 4.45
All nondelinquent coal mines: 3.38

Justice coal mines: 3.3
Justice delinquent coal mines: 3.35
Justice nondelinquent coal mines: 3.49

*****Underground Coal Mines*****
All underground coal mines: 5.41
All delinquent, underground coal mines: 6.5
All nondelinquent, underground coal mines: 4.97

Justice underground coal mines: 5.24
Justice delinquent, underground coal mines: 5.65
Justice nondelinquent, underground coal mines: 2.75

*****Surface Coal Mines*****
All surface coal mines: 1.77
All delinquent, surface coal mines: 2.94
All nondelinquent, surface coal mines: 1.67

Justice surface coal mines: 2.54
Justice delinquent, surface coal mines: 2.74
Justice nondelinquent, surface coal mines: 2.32

*****Facility Coal Mines*****
All facility coal mines: 3.77
All delinquent, facility coal mines: 3.78
All nondelinquent, facility coal mines: 3.72

Justice facility coal mines: 3.3
Justice delinquent, facility coal mi

## Violation analysis at delinquent mines
What kinds of violations are they getting. Why should people care? Are most S&S? Are most likely to cause an accident?

### How many delinquent violations have there been since 2000

In [427]:
del_violations = violations[violations['delinquent'] == 1]
print(del_violations['VIOLATION_NO'].nunique())

171838


## Justice delinquent violations

In [534]:
justice_viols = violations[violations['CONTROLLER_ID'].isin(['0091855','C04355'])]
justice_delinquencies = delinquency_data[delinquency_data['dba_ctrlr_id'].isin(['0091855','C04355'])]
justice_earliest_series = justice_delinquencies.groupby('mine_id')['delinquent_date'].agg('min')

justice_viols['justice_delinquent_date'] = justice_viols['MINE_ID'].map(justice_earliest_series)
justice_viols['justice_delinquent'] = np.where(justice_viols['VIOLATION_ISSUE_DT'] > justice_viols['justice_delinquent_date'],1,0)

/Users/akanik/.virtualenvs/code-FVmSw4CU/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/akanik/.virtualenvs/code-FVmSw4CU/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [564]:
del_justice_viols = justice_viols[justice_viols['justice_delinquent'] == 1]
print(del_justice_viols.CAL_YR.min())
print(del_justice_viols.CAL_YR.max())



#justice_107a = del_justice_viols[del_justice_viols['SECTION_OF_ACT_1'] == '107(a)']

#print(justice_107a[['SIG_SUB','PART_SECTION','SECTION_OF_ACT_1','LIKELIHOOD','CIT_ORD_SAFE','NEGLIGENCE','CONTROLLER_NAME']])

#print(del_justice_viols.info())
#print(del_justice_viols.VIOLATION_NO.nunique())
#print(del_justice_viols.groupby(['CIT_ORD_SAFE','SECTION_OF_ACT'])['VIOLATION_NO'].nunique())
#print(del_justice_viols.groupby(['CIT_ORD_SAFE','SECTION_OF_ACT_1'])['VIOLATION_NO'].nunique())
#print(del_justice_viols.groupby('PART_SECTION')['VIOLATION_NO'].nunique().reset_index().sort_values('VIOLATION_NO',ascending=False))


2009.0
2019.0


### How many delinquent violations of different types have there been since 2000

In [495]:
#print(del_violations.columns)
del_viol_by_type = del_violations.groupby('SECTION_OF_ACT_1')['VIOLATION_NO'].nunique().reset_index()
print(del_viol_by_type)
#print(del_violations.info())



  SECTION_OF_ACT_1  VIOLATION_NO
0           103(j)            11
1           103(k)           817
2           104(a)        161665
3           104(b)          3261
4        104(d)(1)          1958
5        104(d)(2)          1487
6        104(g)(1)          1397
7           107(a)           452
8           314(b)           790


### What types of delinquent violation negligence have been reported

In [266]:
del_violations_by_neg = del_violations.groupby('NEGLIGENCE')['VIOLATION_NO'].nunique()
del_violations_by_likelihood = del_violations.groupby('LIKELIHOOD')['VIOLATION_NO'].nunique()
del_violations_by_sns = del_violations.groupby('SIG_SUB')['VIOLATION_NO'].nunique()
print(del_violations_by_neg)
print(del_violations_by_likelihood)
print(del_violations_by_sns)

NEGLIGENCE
HighNegligence     14798
LowNegligence      23428
ModNegligence     127759
NoNegligence         190
Reckless             344
Name: VIOLATION_NO, dtype: int64
LIKELIHOOD
Highly            1246
NoLikelihood      9741
Occurred           297
Reasonably       49078
Unlikely        106156
Name: VIOLATION_NO, dtype: int64
SIG_SUB
N    116096
Y     50444
Name: VIOLATION_NO, dtype: int64


## How many delinquent injuries have there been, of different kinds, since 2000

In [244]:
del_injuries = injuries[injuries['delinquent'] == 1]
fatal_del_injuries = del_injuries[del_injuries['DEGREE_INJURY_CD'] == '01']
serious_injuries = del_injuries[del_injuries['DEGREE_INJURY_CD'].isin(['03','04','05','01','02'])]

print('Serious injuries: ' + str(len(serious_injuries)))
print(del_injuries.groupby(['DEGREE_INJURY','DEGREE_INJURY_CD'])['DOCUMENT_NO'].nunique())

Serious injuries: 4002
DEGREE_INJURY                   DEGREE_INJURY_CD
ALL OTHER CASES (INCL 1ST AID)  10                    37
DAYS AWAY FROM WORK ONLY        03                  2741
DAYS RESTRICTED ACTIVITY ONLY   05                   796
DYS AWY FRM WRK & RESTRCTD ACT  04                   394
FATALITY                        01                    23
NO DYS AWY FRM WRK,NO RSTR ACT  06                  1535
OCCUPATNAL ILLNESS NOT DEG 1-6  07                   267
PERM TOT OR PERM PRTL DISABLTY  02                    48
Name: DOCUMENT_NO, dtype: int64


## Spend some time investigating how the Debt By Age doc is compiled from the Assessed Violations dataset
I'm pretty sure you need to group assessed violations on at least `ASSESS_CASE_NO` and `EVENT_NO`. I've added `VIOLATOR_ID` to add clarity and because I don't think you should have a single delinquency record that applies to multiple violators. Defies all we know about delinquencies.

In [462]:
ass_viol_grouped = ass_violations.groupby(['ASSESS_CASE_NO','EVENT_NO','VIOLATOR_ID']).agg({'VIOLATION_NO':'nunique',
                                                                                           'OCCURRENCE_DT':'nunique',
                                                                                           'PROPOSED_PENALTY_AMT':'sum',
                                                                                           'CURRENT_ASSESSMENT_AMT':'sum',
                                                                                           'MINEACT_INTEREST_AMT':'sum',
                                                                                           'EXLATE_INTEREST_AMT':'sum',
                                                                                           'PAID_PROPOSED_PENALTY_AMT':'sum',
                                                                                           'PAID_MINEACT_INTEREST_AMT':'sum',
                                                                                           'PAID_EXLATE_INTEREST_AMT':'sum',
                                                                                           'DELINQUENT_DT':'nunique'}).reset_index()

ass_viol_grouped = ass_viol_grouped[ass_viol_grouped['DELINQUENT_DT'] > 0]
ass_viol_grouped.to_csv(data_dir + 'analysis/ass-violations-grouped.csv')

In [494]:
print(del_data_joined[del_data_joined['ass_case_nbr'] == '000183984'])

     catod_oper_id catod_ctrlr_id catod_ctrlr_start_dt catod_ctrlr_end_dt  \
5213       0099325         C14751           2009-02-27         2009-09-13   

              catod_oper_nm ass_case_nbr    mine_id delinquent_date  \
5213  Nine Mile Mining Inc.    000183984  4407252.0      2009-06-06   

      ending_balance  curr_mine_nm  ... curr_stat_dt curr_ctrlr_id  \
5213          342.01  Nine Mile #1  ...   16-10-2018       0091855   

                                 curr_ctrlr_nm curr_oper_id  \
5213  Jillean L  Justice; James C  Justice lll      0099325   

                curr_oper_nm state_abbr fips_cnty_nm curr_ownr_beg_dt  \
5213  Nine Mile Mining, Inc.         VA         Wise       26-01-2011   

     dba_ctrlr_id                              dba_ctrlr_nm  
5213      0091855  James C  Justice lll; Jillian L  Justice  

[1 rows x 23 columns]


In [492]:
#print(ass_violations.columns)
print(ass_violations[ass_violations['ASSESS_CASE_NO'] == '000149549'])
#print(ass_violations[ass_violations['ASSESS_CASE_NO'] == '00035169']['DELINQUENT_DT'].max())
#print(ass_violations.info(verbose=True, null_counts=True))

        EVENT_NO  VIOLATION_NO    MINE_ID VIOLATOR_ID       VIOLATOR_NAME  \
1215447  4230392     7502862.0  1518994.0     0051884  Liggett Mining LLC   

        VIOLATOR_TYPE_CD COAL_METAL_IND OFFICE_CD ASSESS_CASE_NO  \
1215447         Operator              C     C0702      000149549   

        PRIMARY_ACTION_CD  ... MINE_SIZE_POINTS CONTROLLER_HRS  \
1215447            104(a)  ...              9.0            0.0   

        CONTROLLER_PRODUCTION_AMT SIZE_OF_CONTROLLING_ENTITY  \
1215447                 1061521.0                  1061521.0   

        CONTROLLER_SIZE_POINTS DELINQUENT_DT HISTORY_START_DT HISTORY_END_DT  \
1215447                    8.0    2009-12-29       05/10/2006     06/05/2007   

        VIOLATOR_START_DT VIOLATOR_END_DT  
1215447        05/10/2006      09/09/9999  

[1 rows x 58 columns]


In [493]:
print(violations[violations['VIOLATION_NO'] == 7502862])

       EVENT_NO INSPECTION_BEGIN_DT INSPECTION_END_DT  VIOLATION_NO  \
656120  4230392          04/11/2007        06/30/2007     7502862.0   

       CONTROLLER_ID   CONTROLLER_NAME VIOLATOR_ID       VIOLATOR_NAME  \
656120        C12998  Stanley R  Ditty     0051884  Liggett Mining LLC   

       VIOLATOR_TYPE_CD    MINE_ID  ... LAST_ACTION_CD LAST_ACTION_DT  \
656120         Operator  1518994.0  ...       Treasury     04/05/2012   

             DOCKET_NO DOCKET_STATUS_CD CONTESTED_IND CONTESTED_DT  \
656120  KENT 2008-1050         Accepted             Y   05/21/2008   

        VIOLATOR_VIOLATION_CNT  VIOLATOR_INSPECTION_DAY_CNT  delinquent_date  \
656120                    51.0                         73.0       2009-12-29   

        delinquent  
656120           0  

[1 rows x 63 columns]


In [481]:
print(delinquency_data.columns)
justice_viol_inspection_no = violations[violations['CONTROLLER_ID'].isin(['C04355','0091855'])]['VIOLATION_NO'].unique()
justice_ass_viol_cases = ass_violations[ass_violations['VIOLATION_NO'].isin(justice_viol_inspection_no)]
justice_ass_grouped = justice_ass_viol_cases.groupby(['ASSESS_CASE_NO','EVENT_NO','VIOLATOR_ID']).agg({'VIOLATION_NO':'nunique',
                                                                                           'OCCURRENCE_DT':'nunique',
                                                                                           'PROPOSED_PENALTY_AMT':'sum',
                                                                                           'CURRENT_ASSESSMENT_AMT':'sum',
                                                                                           'MINEACT_INTEREST_AMT':'sum',
                                                                                           'EXLATE_INTEREST_AMT':'sum',
                                                                                           'PAID_PROPOSED_PENALTY_AMT':'sum',
                                                                                           'PAID_MINEACT_INTEREST_AMT':'sum',
                                                                                           'PAID_EXLATE_INTEREST_AMT':'sum',
                                                                                           'DELINQUENT_DT':'nunique'}).reset_index()

justice_del = delinquency_data[delinquency_data['dba_ctrlr_id'].isin(['C04355','0091855'])]

ass_viol_grouped_case_nos = ass_violations[['ASSESS_CASE_NO']].drop_duplicates()
justice_ass_grouped_case_nos = justice_ass_grouped[['ASSESS_CASE_NO']].drop_duplicates()
justice_del = justice_del[['ass_case_nbr']].drop_duplicates()

justice_del_ass_join = justice_del.merge(justice_ass_grouped_case_nos, how='left', left_on='ass_case_nbr', right_on='ASSESS_CASE_NO')
justice_del_ass_join_all = justice_del_ass_join.merge(ass_viol_grouped_case_nos, 
                                                      how='left', 
                                                      left_on='ass_case_nbr', 
                                                      right_on='ASSESS_CASE_NO',
                                                     suffixes=('_justice','_all'))
justice_del_ass_join_all.to_csv(data_dir + 'analysis/justice/justice-ass-del-join-check.csv')

#justice_ass_viol_cases.to_csv(data_dir + 'analysis/justice/justice-viols-from-ass-viols.csv')
#justice_ass_grouped.to_csv(data_dir + 'analysis/justice/justice-ass-viols-grouped.csv')
#print(len(justice_ass_grouped))

Index(['ass_case_nbr', 'mine_id', 'violator_id', 'violator_nm',
       'violator_type', 'dba_ctrlr_id', 'dba_ctrlr_nm', 'age_cateogry',
       'delinquent_date', 'delinquent_type', 'ending_balance', 'year',
       'month'],
      dtype='object')
